In [3]:
%pylab
from glob import glob
import h5py
plt.clf()
fig, axes = plt.subplots(1,2,figsize=(8,3),sharex=True)
dirs = sorted(glob("/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output*"))

SFEs = []
for d in dirs:
    F = h5py.File(sorted(glob(d+"/snapshot*.hdf5"))[-1])
    SFEs.append(np.sum(F["PartType5"]["Masses"])/1e-5)

dirs = sorted(zip(SFEs,dirs))[::-1]

labels = {"NORMAL": "IMFMEAN",
          "KIM": "IMFMED",
          "IMF100": "POISSON100",
          "IMF1000": "POISSON1000",
          "KRUMHOLZ": "IMFMEAN+K18",
          #"SS": "SINGLESTAR",
          #"heating": " (Heating)",
          #"momentum": " (Momentum)"
         }

#forbidden = "SS"
i = 0
for sfe, d in dirs:
    print(d)
    if "SS" in d: continue
    flags = d.split("output")[-1].split("_")[1:]
    if len(flags)>1: continue
    snaps = sorted(glob(d+"/snapshot*.hdf5"))
    times = []
    mstar = []
    light_to_mass = []
    for s in snaps:
        F = h5py.File(s)
        times.append(F["Header"].attrs["Time"]*979)
        if "PartType5" in F.keys():
            m = np.sum(F["PartType5"]["Masses"])
            mstar.append(m)
            if "IMF100" in flags:
                light_to_mass.append(np.array(F["PartType5"]["OStarNumber"]).sum() / (m/1e-8))
            elif "IMF1000" in flags:
                light_to_mass.append(np.array(F["PartType5"]["OStarNumber"]).sum() / (m/1e-7))
            elif "KIM" in flags:
                logx = np.log(m/1e-10)
                light_to_mass.append(0.001047 * np.exp(6.8617*np.power(logx,6)/(4322.07 + np.power(logx,6))))
            else:
                light_to_mass.append(1.)
        else:
            mstar.append(0)
            light_to_mass.append(np.nan)
        
        F.close()
    #print(times)
    label = ("".join([labels[f] for f in flags]) if not (("heating" in flags) or("momentum" in flags)) else None)
    ls = ("dashed" if "heating" in flags else "dotted" if "momentum" in flags else "solid")
    SFE = np.array(mstar)/1e-5
    color = plt.cm.viridis(i/4.2)
    axes[0].plot(times, SFE, ls=ls, color='black', lw=1.5)
    axes[0].plot(times, SFE, label=r"$\texttt{%s}$"%label, ls=ls, color=color)
    axes[1].plot(times,light_to_mass, ls=ls, color='black', lw=1.5)
    axes[1].plot(times, light_to_mass, label=r"$\texttt{%s}$"%label, ls=ls, color=color)
    
    i += 1
axes[0].set_xlim(0,7)
axes[0].legend()
axes[1].set_ylim(0,2)
axes[1].set_yticks([0,1,2])
#axes[1].set_yscale('log')
axes[0].set_xlabel("Time (Myr)")
axes[1].set_xlabel("Time (Myr)")
axes[0].set_ylabel(r"$M_\star/M_{total}$")
axes[1].set_ylabel(r"$\frac{L_{tot}}{M_\star}/\left\langle\frac{L_\star}{M_\star}\right\rangle_{IMF}$")
axes[0].legend(labelspacing=0)
#axes[0].set_yscale('log')
plt.savefig("./SFE_and_L_over_M.pdf",bbox_inches='tight')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/mike/.conda/envs/idp/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['axes']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM_KRUMHOLZ_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_NORMAL_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KRUMHOLZ_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_IMF100_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM_KRUMHOLZ_momentum
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_IMF1000_heating
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM_KRUMHOLZ
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM_momentum
/media/mike/STORAGEBOYE/mgrudic/kim2018/M1e5_R20_S0_T1_B0.01_Res100_n2_sol1_1/output_KIM
/m

In [16]:
x = np.linspace(0,5*np.log(10),1000)

plt.plot(x,0.001047 * np.exp(6.8617*np.power(x,6)/(4322.07 + np.power(x,6)))); plt.show()